In [ ]:
# Colab cell: montar Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Ruta recomendada para trabajar (ajusta si quieres otra carpeta)
%env DRIVE_ROOT=/content/drive/MyDrive/colab_rl_project
!mkdir -p $DRIVE_ROOT
!ls -la $DRIVE_ROOT

Mounted at /content/drive
env: DRIVE_ROOT=/content/drive/MyDrive/colab_rl_project
total 0


In [ ]:
%cd /content
!git clone https://github.com/Naisito/ProyectoRL.git proyecto_rl
!cd proyecto_rl

/content
Cloning into 'proyecto_rl'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 74 (delta 25), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 116.35 KiB | 5.29 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [ ]:
# Colab bash cell
%cd /content/proyecto_rl
!pwd
!ls -la

/content/proyecto_rl
/content/proyecto_rl
total 44
drwxr-xr-x 6 root root  4096 Jan 15 19:55 .
drwxr-xr-x 1 root root  4096 Jan 15 19:55 ..
drwxr-xr-x 2 root root  4096 Jan 15 19:55 configs
drwxr-xr-x 8 root root  4096 Jan 15 19:55 .git
drwxr-xr-x 2 root root  4096 Jan 15 19:55 modelos
-rw-r--r-- 1 root root 11939 Jan 15 19:55 readme.md
-rw-r--r-- 1 root root   343 Jan 15 19:55 requirements.txt
-rw-r--r-- 1 root root   986 Jan 15 19:55 setup_env.ps1
drwxr-xr-x 3 root root  4096 Jan 15 19:55 src


In [ ]:
# Colab bash cell: pip setup seguro

# Instalar dependencias del sistema para box2d-py
!apt-get update
!apt-get install -y swig build-essential

# 1) actualizar pip
!python -m pip install -U pip setuptools wheel

# 2) forzar reinstalación de numpy (versión fijada para evitar uninstall-no-record)
!python -m pip install --force-reinstall --no-deps numpy==1.26.4

# 3) instalar el resto desde requirements.txt
!python -m pip install -r requirements.txt

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://cli.github.com/packages stable/main amd64 Packages [354 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,968 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,600 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,411 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [69.2 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports/universe amd64 Packages [37.2 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports/main amd64 Packages [83.9 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy InRelease

In [ ]:
# Colab cell: detecta CPU/GPU, propone n_envs y num_threads; exporta variables de entorno
import os, math, multiprocessing, torch

cpu_count = os.cpu_count() or 1
gpu_available = torch.cuda.is_available()
gpu_name = torch.cuda.get_device_name(0) if gpu_available else "no-gpu"

# heurística segura:
# - si GPU potente (A100), proponemos preferir GPU y aumentar n_envs (8-16) si CPU lo permite.
# - elegimos n_envs = min(16, max(1, cpu_count//2))
proposed_n_envs = min(16, max(1, cpu_count // 2))
# Ensure not too small for tiny CPUs:
proposed_n_envs = max(1, proposed_n_envs)

# num_threads por proceso para evitar oversubscription:
if proposed_n_envs > 0:
    proposed_num_threads = max(1, cpu_count // proposed_n_envs)
else:
    proposed_num_threads = max(1, cpu_count)

print(f"CPU logical cores: {cpu_count}")
print(f"GPU disponible: {gpu_available} ({gpu_name})")
print(f"Propuesta: --n-envs {proposed_n_envs}  --num-threads {proposed_num_threads}")

# Exportar variables para BLAS/MKL/OpenMP y PyTorch
os.environ["OMP_NUM_THREADS"] = str(proposed_num_threads)
os.environ["MKL_NUM_THREADS"] = str(proposed_num_threads)
os.environ["OPENBLAS_NUM_THREADS"] = str(proposed_num_threads)
os.environ["VECLIB_MAXIMUM_THREADS"] = str(proposed_num_threads)

import torch
torch.set_num_threads(proposed_num_threads)

# Imprime confirmación
print("Variables de hilo exportadas y torch.set_num_threads aplicado.")

CPU logical cores: 44
GPU disponible: False (no-gpu)
Propuesta: --n-envs 16  --num-threads 2
Variables de hilo exportadas y torch.set_num_threads aplicado.


In [ ]:
import os
import shutil

# 1. Definir rutas
drive_results = "/content/drive/MyDrive/colab_rl_project/results"
local_results = "/content/proyecto_rl/results" # Asumo que tu script guarda aquí por defecto

# 2. Crear carpeta en Drive si no existe
os.makedirs(drive_results, exist_ok=True)

# 3. TRUCO DE MAGIA: Enlace Simbólico
# Si el script intenta guardar en local, en realidad estará escribiendo en Drive.
if os.path.exists(local_results):
    # FIX: Comprobar si es un enlace simbólico antes de borrar
    if os.path.islink(local_results):
        os.unlink(local_results)
    else:
        shutil.rmtree(local_results) # Borrar carpeta local si ya existe vacía

os.symlink(drive_results, local_results)
print(f"Enlace creado: {local_results} -> apuntando a -> {drive_results}")

# 4. Ejecutar entrenamiento DIRECTO
# Quitamos 'tee' para que la barra de progreso (tqdm) detecte la terminal y se muestre correctamente.
print("Iniciando entrenamiento...")

!python -u -m src.train \
  --config configs/optuna_mejores.yaml \
  --device cuda \
  --n-envs 16 \
  --num-threads 2 \
  --total-timesteps 5000000

Enlace creado: /content/proyecto_rl/results -> apuntando a -> /content/drive/MyDrive/colab_rl_project/results
Iniciando entrenamiento...
/usr/local/lib/python3.12/dist-packages/google/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
📁 Run dir: results/ppo_carracing_5m/20260111-220000
2026-01-11 22:00:00,024 | INFO    | root | Inicio del run
2026-01-11 22:00:00,025 | INFO    | root | Device: cuda
/usr/local/lib/python3.12/dist-packages/google/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/usr/local/lib/python3.12/dist-packages/g

In [ ]:
%tensorboard --logdir results

UsageError: Line magic function `%tensorboard` not found.
